In [1073]:
import pandas as pd

In [1074]:
# # read the excel file and get all the sheets
# xls = pd.ExcelFile('data.xlsx')
# # get the names of all sheets
# sheet_names = xls.sheet_names

# print("Sheet names:", sheet_names)

In [1075]:
# all_data = pd.DataFrame()

In [1076]:
# # Loop through each sheet
# for sheet_name in sheet_names:
#     # Read the sheet into a DataFrame
#     df = pd.read_excel(xls, sheet_name=sheet_name)
#     # Add a new column with the sheet name
#     df['category'] = sheet_name  # No need to add quotes around the sheet name
#     # Concatenate the dataframes
#     all_data = pd.concat([all_data, df], ignore_index=True)


# all_data = all_data.dropna(how='all', axis=1)  # Drop columns that are completely empty

In [1077]:
# # add a product_id column at the beginning
# all_data.insert(0, 'product_id', range(1, len(all_data) + 1))

# all_data = all_data.rename(columns=lambda x: x.strip())  # Remove leading and trailing spaces from column names

# all_data

In [1078]:
# # move the 'category' column next to the name column
# category_col = all_data.pop('category')
# all_data.insert(2, 'category', category_col)

# all_data

In [1079]:
# # save the final DataFrame to a new csv file
# all_data.to_csv('data.csv', index=False)

In [1080]:
data = pd.read_csv('data.csv')

# data

In [1081]:
# leave data where catogory is "Fruits & Vegetables"
data = data[data['category'] != 'Fruits & Vegetables']

# data

In [1082]:
all_names = data['name']

# if any two names have first word same, add them to a group
groups = {}
for name in all_names:
    first_word = name.split()[0]
    if first_word not in groups:
        groups[first_word] = []
    groups[first_word].append(name)

# for each key in groups, remove the duplicates from the value list
for key, value in groups.items():
    groups[key] = list(set(value))

# add the groups with only one name to a new dictionary 
no_groups = {}
for key, value in groups.items():
    if len(value) == 1:
        no_groups[key] = value
# remove the groups with only one name from the original dictionary
for key in no_groups.keys():
    del groups[key]
    


In [1083]:
# number of elements in no_groups
# len(no_groups)
# no_groups

In [1084]:
# # print the groups
# for key, value in groups.items():
#     print(f"{key}: {value}")

In [1085]:
# for each key in groups, find the longest matching string in the value list
def find_longest_matching_string(strings):
    if not strings:
        return ""
    
    min_words_in_group = 1000
    lss = []
    for s in strings:
        lss.append(s.split())
        min_words_in_group = min(min_words_in_group, len(s.split()))
    
    max_words_matched = 0
    for i in range(min_words_in_group):
        words = []
        for s in lss:
            words.append(s[i])
        if len(set(words)) == 1:
            max_words_matched += 1
        else:
            break
    
    return " ".join(lss[0][:max_words_matched])
# find the longest matching string in each group



In [1086]:
longest_matching_strings = {}
for key, value in groups.items():
    longest_matching_strings[key] = find_longest_matching_string(value)
    
# print the longest matching strings
# for key, value in longest_matching_strings.items():
#     print(f"{key}: {value}")

In [1087]:
# add a new column to the data with the brand name
data.insert(2, 'brand', "NANA")
# data

In [1088]:
# for each key in groups, for each value in value list, find the index of the values in the data and set the brand name to the longest matching string
for key, value in groups.items():
    for v in value:
        indices = data[data['name'] == v].index
        for i in indices:
            data.at[i, 'brand'] = longest_matching_strings[key]
        
# data

In [1089]:

data.loc[data['name'].str.contains("TilSona"), 'brand'] = "TilSona"
data.loc[data['name'].str.contains("Nissin"), 'brand'] = "Nissin"
data.loc[data['name'].str.contains("Puro"), 'brand'] = "Puro Healthy"
data.loc[data['name'].str.contains("Catch"), 'brand'] = "Catch"
data.loc[data['name'].str.contains("Japanese Choice"), 'brand'] = "Japanese Choice"
data.loc[data['name'].str.contains("India Gate"), 'brand'] = "India Gate"
data.loc[data['name'].str.contains("Dlicia Whip"), 'brand'] = "Dlicia"
data.loc[data['name'].str.contains("Jivana Jaggery"), 'brand'] = "Jivana"
data.loc[data['name'].str.contains("Horlicks"), 'brand'] = "Horlicks"
data.loc[data['name'].str.contains("Sugarlite"), 'brand'] = "Sugarlite"

# now remove the rows where brand is "NANA"
data = data[data['brand'] != "NANA"]

In [1090]:
# calculate the number of rows with the same name and same weightInGrams
data['name_weight'] = data['name'] + " " + data['weightInGms'].astype(str)
name_counts = data['name_weight'].value_counts()

# print the name counts
# for name, count in name_counts.items():
#     print(f"{name}: {count}")


In [1091]:
# remove the duplicates in name_weight and keep the first occurrence
data = data.drop_duplicates(subset=['name_weight'], keep='first')

# remove the name_weight column
data = data.drop(columns=['name_weight'])

data['name_weight'] = data['name'] + " " + data['weightInGms'].astype(str)
name_counts = data['name_weight'].value_counts()

# print the name counts
# for name, count in name_counts.items():
#     print(f"{name}: {count}")
    
data = data.drop(columns=['name_weight'])

In [1092]:
# export the data to a new csv file
data.to_csv('data_with_brand.csv', index=False)